In [2]:
import numpy as np
import scipy.stats as stats
import scipy.special as spec
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Good-Toulmin Estimators
---

Or my adventures in pulling extra information from the clonal landscape of scDNA-Seq of tumors cells

For reference on methodology see http://www.stat.yale.edu/~yw562/reprints/species-pnas.pdf as well as Good & Toulmins original paper as well as the paper by Efron & Thisted where they use the k-truncated euler transform of Good & Toulmins original estimator

TODO testcases:

see for instance https://www.rdocumentation.org/packages/preseqR/versions/2.0.1.1/topics/preseqR.rfa.species.accum.curve


In [46]:
def good_toulmin(phi, t):
    """
    The original Good Toulmin Estimator
    """
    ts = (-t)**np.arange(1, phi.size+1)
    return -np.dot(ts, phi)

    
def smoother(i, n, t):
    """
    The binomial smoothing distribution from Orlitsky et al
    using their proposed parametrization from Table 1 (row 2)
    """
    k = np.floor(0.5 * np.log2( (n*(t**2))/(t-1)))
    k = 3
    ds = 1-stats.binom.cdf(i, k, 1/(1+t))
    return ds


def smoothed_good_toulmin(phi, n, t, smf=smoother):
    """
    The smoothed Good Toulmin estimator as proposed by Orlitksy et al
    in PNAS 2016 using the proposed binomial smoothing distribution.
    """
    ts = (-t)**np.arange(1, phi.size+1)
    i = np.arange(1, phi.size +1)
    sm = smf(i, n, t)
    return -np.sum(ts*sm*phi)

In [49]:
# Original Wright Data
"""
wright = np.array([118, 74, 44, 24, 29, 22, 20, 19, 20, 15, 12, 14, 6, 12, 6])
n_samples = 0
for index in range(1, wright.size+1):
    n_samples += index*wright[index]
    

phi = np.array([2,1,1])
print(smoothed_good_toulmin(wright, 2029, 1.01))

t_series = np.arange(1.1,10,step=0.25)
g = []
#print(t_series)
for c in t_series:
    g.append(smoothed_good_toulmin(wright, 2029, c))

u_series = np.array(g)
plt.plot(t_series, u_series)
"""

def et_sm(i, k, t):
    psum = 0
    if i > k: return 0
    for j in range(i+1, k+1):
        psum += spec.comb(k, j)*((t**(k-j))/((1+t)**k))
    return psum

def et_sm_vec(il, n, t):
    k = np.floor(0.5 * np.log2( (n*(t**2))/(t-1)))
    return np.array([et_sm(ii, int(k), t) for ii in il])

n = 10
t = 1.05
i = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
print(et_sm_vec(i, k, t))
print(smoother(i, k, t))

[ 0.48171094  0.11607493  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.        ]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[ 0.48171094  0.11607493  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.        ]
[ 0.48171094  0.11607493  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.        ]
